## 『本次練習內容』
#### 學習如何搭建 Residual Block
####  學習如何搭建Inception-ResNet中的 Inception Block

## 『本次練習目的』
  #### 了解 Residual Block原理
  #### 了解如何結合Inception 與 Residual概念

## Part1

In [1]:
import numpy as np
from keras.models import Model
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras import backend as K
from keras import layers
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Lambda

Using TensorFlow backend.


![Incpeiton](img/ResNet_Structure.png)

## ResNetV1版本的Residual Block

In [2]:
# ResNet Original (2 convolution layer) :
def Residual_block(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2, filters3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)  # weight layer
    x = BatchNormalization(axis=3, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same', name=conv_name_base + '2b')(x)  # weight layer
    x = BatchNormalization(axis=3, name=bn_name_base + '2b')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x

## 參考ResNetV1 搭建 ResNetV2版本的Residual Block

In [3]:
# ResNet full pre-activation :
def Residual_block_v2(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2, filters3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # build Residual_V2 Block
    # BN before each weight layer
    x = BatchNormalization(axis=3, name=bn_name_base + '2a')(input_tensor)
    x = Activation('relu')(x)
    x = Conv2D(filters2, kernel_size, padding='same', name=conv_name_base + '2a')(x)  # weight layer

    # BN before each weight layer
    x = BatchNormalization(axis=3, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)
    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2b')(x)  # weight layer

    # remove Relu
    x = layers.add([x, input_tensor])
    #x = Activation('relu')(x)
    
    return x

## 試試看自己設計一個先壓縮再回放的V2 Block

In [4]:
# reduce and recover filters with Residual-V2
def Residual_block_v2(input_tensor, kernel_size, stage, block, reduec=64, output_size=128):
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # reduce filters with kernel_size=(1,1)
    x = BatchNormalization(axis=3, name=bn_name_base + '2a')(input_tensor)
    x = Activation('relu')(x)
    x = Conv2D(reduec, (1, 1), padding='same', name=conv_name_base + '2a')(x)  # weight layer

    # reduce filters with kernel_size=kernel_size
    x = BatchNormalization(axis=3, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)
    x = Conv2D(reduec, kernel_size, name=conv_name_base + '2b')(x)  # weight layer

    # recover filters with kernel_size=(1,1)
    x = Conv2D(output_size, (1, 1), padding='same', name=conv_name_base + '2a')(x)  # weight layer
    
    # remove Relu
    x = layers.add([x, input_tensor])
    
    return x

## Part2

## Incpetion Block-A

![Incpeiton](img/Inception-ResNet-A.png)

## Incpetion Block-B

![Incpeiton](img/Inception-ResNet-B.png)

## Incpetion Block-C

![Incpeiton](img/Inception-ResNet-C.png)

In [5]:
# padding = 'same', strides = (1,1), let h,w of output keep same as h,w of input 
def Conv2d_bn(x,filters,kernel_size,padding='same',strides=(1, 1),normalizer=True,activation='relu',name=None):
    if name is not None:
        conv_name = name + '_conv'
        bn_name = name + '_bn'
        act_name = name + '_act'
    else:
        conv_name = None
        bn_name = None
        act_name = None
    # 'channels_first' : corresponds to inputs with shape (batch_size, channels, height, width)
    # 'channels_last'  : corresponds to inputs with shape (batch_size, height, width, channels)
    if K.image_data_format() == 'channels_first':  
        bn_axis = 1
    else:
        bn_axis = 3
    x = Conv2D(
            filters, kernel_size,
            strides=strides, padding=padding,
            use_bias=False, name=conv_name)(x)
    if normalizer:
        x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    if activation:
        x = Activation(activation, name=act_name)(x)
    return x

In [ ]:
# ResNet Original (3 convolution layer) :
def Residual_block(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2, filters3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)  # weight layer
    x = BatchNormalization(axis=3, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same', name=conv_name_base + '2b')(x)  # weight layer
    x = BatchNormalization(axis=3, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)  # weight layer
    x = BatchNormalization(axis=3, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x

## 參考上方Residual_block搭建 Inception-ResNet中的Inception Block

In [8]:
def inception_resnet_block(x, scale, block_type, activation='relu'):
    '''scale: scaling factor to scale the residuals (i.e., the output of
            passing `x` through an inception module) before adding them
            to the shortcut branch. Let `r` be the output from the residual branch,
            the output of this block will be `x + scale * r`.(簡單來說就是控制Residual branch的比例)'''
    
    # inception block :
    if block_type == 'Incpetion_Block-A':
        branch_0 = Conv2d_bn(x, 32, 1)
        branch_1 = Conv2d_bn(x, 32, 1)
        branch_1 = Conv2d_bn(branch_1, 32, 3)
        branch_2 = Conv2d_bn(x, 32, 1)
        branch_2 = Conv2d_bn(branch_2, 48, 3)
        branch_2 = Conv2d_bn(branch_2, 64, 3)
        branches = [branch_0, branch_1, branch_2]
    elif block_type == 'Incpetion_Block-B':
        branch_0 = Conv2d_bn(x, 192, 1)
        branch_1 = Conv2d_bn(x, 128, 1)
        branch_1 = Conv2d_bn(branch_1, 160, [1, 7])
        branch_1 = Conv2d_bn(branch_1, 192, [7, 1])
        branches = [branch_0, branch_1]
    elif block_type == 'Incpetion_Block-C':
        branch_0 = Conv2d_bn(x, 192, 1)
        branch_1 = Conv2d_bn(x, 192, 1)
        branch_1 = Conv2d_bn(branch_1, 192, [1, 3])
        branch_1 = Conv2d_bn(branch_1, 192, [3, 1])
        branches = [branch_0, branch_1]
    else:
        raise ValueError('Unknown Inception-ResNet block type. '
                         'Expects "block35", "block17" or "block8", '
                         'but got: ' + str(block_type))
    
    mixed = Concatenate(axis=3)(branches)  # = layers.concatenate(branches, axis=3)
    # assume input shape x = (None, 224, 224, 32)
    # assume block_type == 'Incpetion_Block-A', mixed output shape=(None, 224, 224, 128)
    
    '''確保輸入跟輸出深度相同'''
    # Conv2d_bn(img_input, filters, kernel_size)
    # input=mixed, filters=32, kernel_size=1, output shape=(None, 224, 224, 32), output depth neet to be same as input=32
    up = Conv2d_bn(mixed, K.int_shape(x)[3], 1, activation=None)
    
    ''' 導入Residual Block殘差結構，並給予權重scale '''
    # Lambda(function, output_shape=None, mask=None, arguments=None)
    # inputs=[x, up], means input[0]=x, input[1]=up, so Residual Block x = x + up*(scale)
    x = Lambda(lambda inputs, scale: inputs[0] + inputs[1] * scale,
               output_shape = K.int_shape(x)[1:],     # (224, 224, 32)
               arguments = {'scale': scale})([x, up]) 
                                                      
    if activation is not None:
        x = Activation(activation)(x)
    return x

In [9]:
img_input = Input(shape=(224,224,32))  
x = inception_resnet_block(img_input, 0.1, 'Incpetion_Block-A', activation='relu')
print(x)  # shape=(None, 224, 224, 32)

Tensor("activation_14/Relu:0", shape=(None, 224, 224, 32), dtype=float32)


## 測試

In [10]:
img_input = Input(shape=(224,224,32))  
x = inception_resnet_block(img_input, 0.1, 'Incpetion_Block-B', activation='relu')
print(x)  # shape=(None, 224, 224, 32)

Tensor("activation_19/Relu:0", shape=(None, 224, 224, 32), dtype=float32)


In [11]:
img_input = Input(shape=(224,224,32))  
x = inception_resnet_block(img_input, 0.1, 'Incpetion_Block-C', activation='relu')
print(x)  # shape=(None, 224, 224, 32)

Tensor("activation_24/Relu:0", shape=(None, 224, 224, 32), dtype=float32)


## 嘗試導入Inception Block到 Vgg_Inception中